In [3]:
import gen_dataset_810 as mod_810fr
# importer datetime pour typer les donnees de dates des chaines et des videos
import datetime
import pandas as pd

# Définition des dates limites pour les filtres de https://s01.810.fr
datemin_2022 = datetime.datetime(2022, 1, 1, 0, 0, 0)
datemax_2022 = datetime.datetime(2022, 4, 1, 0, 0, 0)
datemin_2021 = datetime.datetime(2021, 1, 1, 0, 0, 0)

# Récupération des fichiers générés à partir des API YouTube
folder_candidats = './data_sources/stats_videos/2022_03_08/'
# Création d'un datafram qui agrège les fichiers
dt_candidats = mod_810fr.mb_ds()
# Génération d'un dataframe de vidéos et d'un dataframe de chaîne (exemple dans les blocs qui suivent)
videos_candidats_toutes, chaines_candidats_toutes  = dt_candidats.get_full_data_candidat(folder = folder_candidats)

# Création du dataframe de vidéos, spécial 2021
videos_candidats_2021 = videos_candidats_toutes[videos_candidats_toutes['dateVideo'] > datemin_2021]
videos_candidats_2021 = videos_candidats_2021[videos_candidats_2021['dateVideo'] < datemin_2022]

# On ne conserve que les lignes associées aux candidat.e.s officiel.le.s
liste_chaines_candidats_officiels = ["Nathalie Arthaud", "Jean-Luc Mélenchon", "Jean Lassalle", "Anne Hidalgo", "Marine Le Pen", "Valérie Pécresse", "Nicolas Dupont-Aignan", "Yannick Jadot","Avec vous", "Eric Zemmour"]
# Filtrage des vidéos de l'année 2021
boolean_series_2021 = videos_candidats_2021.candidat.isin(liste_chaines_candidats_officiels)
videos_candidats_2021 = videos_candidats_2021[boolean_series_2021]


dossier_output = './data_output/'
activityfolder_csv = 'activity-csv/'

Imported 32 channel data


In [4]:
import retraitement_data_csa as csa

# Init CSA data
csa_octobre_21 = "./data_sources/donnees_csa/synthese_CSA_octobre_2021.csv"
csa_novembre_21 = "./data_sources/donnees_csa/synthese_CSA_novembre_2021.csv"
csa_decembre_21 = "./data_sources/donnees_csa/synthese_CSA_decembre_2021.csv"

dtf_csa = csa.init_csa_dataframe(lst_fichiers = [csa_octobre_21, csa_novembre_21, csa_decembre_21])
dtf_csa_augmente = csa.create_filtered_csa_dtf(dtfcsa = dtf_csa)

videos_entre_oct_et_dec = videos_candidats_2021[videos_candidats_2021['dateVideo'] > datetime.datetime(2021,10,1,0,0,0)]
# videos_entre_oct_et_dec.to_csv('videos_postees_apres_octobre.csv', index = True)

# Dessine un barchart de temps de parole par type de media
def barres_duree_media_croise(dtfcsa, dtfyt, titre = "") :
    lst_lbl = []
    lst_values = []
    for media in dtfcsa.media.unique() :
        subdtf = dtfcsa[dtfcsa['media'] == media]
        lst_lbl.append(media)
        lst_values.append(csa.somme_temps(subdtf=subdtf))

    lst_lbl.append('YouTube')
    lst_values.append(dtfyt["dureeVideo"].sum().total_seconds() / 60)

    d = {'media' : lst_lbl, 'duree' : lst_values}
    d
    df = pd.DataFrame(d)
    df = df.sort_values(by=['duree'], ascending=False)

    return df


dtf_media = barres_duree_media_croise(dtfcsa=dtf_csa_augmente, dtfyt= videos_entre_oct_et_dec, titre="Durée de parole cumulée des candidats par media, du 1er Octobre 2021 ay 31 Décembre 2021\n")

dtf_media.to_csv(dossier_output + activityfolder_csv + 'media.csv')

Données CSA importées - 6656 lignes
